In [2]:
import os
import sys
import numpy as np
from collections import defaultdict
import gmpy2
import copy

inputfilepath = 'C:\EMOLGUI\A_O_C\Day15.txt'

In [3]:
with open(inputfilepath,'r') as f:
    readingList = []
    min_x = 0
    max_x = 0
    for line in f:
        lineItems = line.rstrip().split(':')
        sensor_x = int(lineItems[0].split(',')[0].split('=')[1])
        sensor_y = int(lineItems[0].split(',')[1].split('=')[1])
        beacon_x = int(lineItems[1].split(',')[0].split('=')[1])
        beacon_y = int(lineItems[1].split(',')[1].split('=')[1])
        sensingDistance = abs(beacon_x - sensor_x) + abs(beacon_y - sensor_y)
        min_x = min(min_x, sensor_x - sensingDistance)
        max_x = max(max_x, sensor_x + sensingDistance)
        reading = {'sensor':(sensor_x,sensor_y), 'beacon':(beacon_x,beacon_y), 'distance':sensingDistance}
        readingList.append(reading)

In [39]:
min_x, max_x, len(readingList), readingList

(-1754016,
 4901396,
 24,
 [{'sensor': (3988693, 3986119),
   'beacon': (3979063, 3856315),
   'distance': 139434},
  {'sensor': (1129181, 241785),
   'beacon': (1973630, -98830),
   'distance': 1185064},
  {'sensor': (2761889, 2453622),
   'beacon': (2803715, 2643139),
   'distance': 231343},
  {'sensor': (3805407, 3099635),
   'beacon': (3744251, 2600851),
   'distance': 559940},
  {'sensor': (3835655, 3999745),
   'beacon': (3979063, 3856315),
   'distance': 286838},
  {'sensor': (3468377, 3661078),
   'beacon': (3979063, 3856315),
   'distance': 705923},
  {'sensor': (1807102, 3829998),
   'beacon': (2445544, 3467698),
   'distance': 1000742},
  {'sensor': (2774374, 551040),
   'beacon': (1973630, -98830),
   'distance': 1450614},
  {'sensor': (2004588, 2577348),
   'beacon': (2803715, 2643139),
   'distance': 864918},
  {'sensor': (2949255, 3611925),
   'beacon': (2445544, 3467698),
   'distance': 647938},
  {'sensor': (2645982, 3991988),
   'beacon': (2445544, 3467698),
   'dista

In [40]:
y_check = 2_000_000
#y_check = 10
covered_segments_x = []
covered_segments_points = []
for reading in readingList:
    distance = reading['distance']
    sensor_x, sensor_y = reading['sensor']
    height_diff = abs(y_check - sensor_y)
    horizontal_spread = distance - height_diff
    if horizontal_spread >= 0:
        segment = [sensor_x - horizontal_spread, sensor_x + horizontal_spread]
        covered_segments_x.append(segment)
        covered_segments_points.append([sensor_x - horizontal_spread, 'start'])
        covered_segments_points.append([sensor_x + horizontal_spread, 'end'])
        print(f'Height difference {height_diff} is allows than sensor range {distance} to bring a horizontal spread {horizontal_spread}')
    else:
        print(f'Height difference {height_diff} is larger than sensor range {distance}')
sorted_covered_segments_point = sorted(covered_segments_points)


Height difference 1986119 is larger than sensor range 139434
Height difference 1758215 is larger than sensor range 1185064
Height difference 453622 is larger than sensor range 231343
Height difference 1099635 is larger than sensor range 559940
Height difference 1999745 is larger than sensor range 286838
Height difference 1661078 is larger than sensor range 705923
Height difference 1829998 is larger than sensor range 1000742
Height difference 1448960 is allows than sensor range 1450614 to bring a horizontal spread 1654
Height difference 577348 is allows than sensor range 864918 to bring a horizontal spread 287570
Height difference 1611925 is larger than sensor range 647938
Height difference 1991988 is larger than sensor range 724728
Height difference 880445 is larger than sensor range 579065
Height difference 231046 is allows than sensor range 552006 to bring a horizontal spread 320960
Height difference 882560 is larger than sensor range 488338
Height difference 720288 is larger than se

In [41]:
coverage_level = 0
clean_sorted_covered_segments_point = []
for point in sorted_covered_segments_point:
    if point[1] == 'start':
        if coverage_level == 0:
            clean_sorted_covered_segments_point.append(point)
        coverage_level += 1
    if point[1] == 'end':
        coverage_level -= 1
        if coverage_level == 0:
            clean_sorted_covered_segments_point.append(point)


In [42]:
clean_sorted_covered_segments_point


[[-836454, 'start'], [1163688, 'end'], [1163688, 'start'], [4345102, 'end']]

In [45]:
clean_positions_raw = clean_sorted_covered_segments_point[-1][0] - clean_sorted_covered_segments_point[0][0] + 1
clean_positions_raw

5181557

In [46]:
for reading in readingList:
    beacon_x, beacon_y = reading['beacon'][0], reading['beacon'][1]
    if beacon_y == y_check:
        print(beacon_x, beacon_y)

1163688 2000000
1163688 2000000
1163688 2000000
1163688 2000000


The number of positions is equal to the range of overlapped segments minus the number of beacons that exist in that segment overlap.

#### QUESTION 2

In [39]:
import os
import sys
import numpy as np
from collections import defaultdict
import gmpy2
import copy

inputfilepath = 'C:\EMOLGUI\A_O_C\Day15.txt'

#inputfilepath = 'C:\EMOLGUI\A_O_C\Day15.sample.txt'

In [40]:
with open(inputfilepath,'r') as f:
    readingList = []
    min_x = 0
    max_x = 0
    for line in f:
        lineItems = line.rstrip().split(':')
        sensor_x = int(lineItems[0].split(',')[0].split('=')[1])
        sensor_y = int(lineItems[0].split(',')[1].split('=')[1])
        beacon_x = int(lineItems[1].split(',')[0].split('=')[1])
        beacon_y = int(lineItems[1].split(',')[1].split('=')[1])
        sensingDistance = abs(beacon_x - sensor_x) + abs(beacon_y - sensor_y)
        min_x = min(min_x, sensor_x - sensingDistance)
        max_x = max(max_x, sensor_x + sensingDistance)
        reading = {'sensor':(sensor_x,sensor_y), 'beacon':(beacon_x,beacon_y), 'distance':sensingDistance}
        readingList.append(reading)

In [41]:
def checkCoverage(reading, range_x, range_y):
    coverage = []
    sensor_x = reading['sensor'][0]
    sensor_y = reading['sensor'][1]
    sensor_distance = reading['distance']
    corners = ['top_left', 'bottom_left', 'top_right', 'bottom_right']

    for x in range_x:
        for y in range_y:
            corner = corners.pop(0)
            distance = abs(sensor_x - x) + abs(sensor_y - y)
            if distance < sensor_distance:
                coverage.append(corner)
    return coverage



In [42]:
def checkPieceFullyCovered(readingList,range_x,range_y):
    sensor_coverage = []
    corners = ['top_left', 'bottom_left', 'top_right', 'bottom_right']
    cornerOppositeSidesMap = {'top_left':['bottom','right'], 'top_right':['bottom','left'],'bottom_left':['top', 'right'],'bottom_right':['top','left']}
    for reading in readingList:
        sensor_coverage.append(checkCoverage(reading, range_x, range_y))

    triCornerCheckList = set()
    duoCornerCheckList = set()
    for coverageCorners in sensor_coverage:
        if len(coverageCorners) == 4:
            return True
        elif len(coverageCorners) == 3:
            for corner in corners:
                if corner not in coverageCorners:
                    triCornerCheckList.add(corner)
                    for side in cornerOppositeSidesMap[corner]:
                        duoCornerCheckList.add(side)
        elif len(coverageCorners) == 2:
            if 'top_left' in coverageCorners and 'top_right' in coverageCorners:
                duoCornerCheckList.add('top')
            if 'top_left' in coverageCorners and 'bottom_left' in coverageCorners:
                duoCornerCheckList.add('left')
            if 'top_right' in coverageCorners and 'bottom_right' in coverageCorners:
                duoCornerCheckList.add('right')
            if 'bottom_left' in coverageCorners and 'bottom_right' in coverageCorners:
                duoCornerCheckList.add('bottom')

    else:
        if ('top_left' in triCornerCheckList and 'bottom_right' in triCornerCheckList) or ('top_right' in triCornerCheckList and 'bottom_left' in triCornerCheckList) or (len(duoCornerCheckList) == 4):
            return True
        else:
            return False


In [43]:
def searchUncovered(readingList,range_x,range_y,size_range=0):
    if size_range == 0:
        size_range = int((range_x[1] - range_x[0] + 1)*(range_y[1] - range_y[0] + 1))
    
    candidate_list = []
    
    print(f'Checking piece of size {size_range} and ranges {range_x} and {range_y}')
    
    if checkPieceFullyCovered(readingList,range_x,range_y):
        return False
    
    if size_range == 1:
        print(f'NON COVERED RANGE AT {range_x} {range_y}')
        return True

    midpoint_x = int((range_x[0] + range_x[1])/2)
    midpoint_y = int((range_y[0] + range_y[1])/2)

    subpart_x_list = []
    subpart_x_list.append((range_x[0],midpoint_x))
    subpart_x_list.append((midpoint_x,range_x[1]))

    subpart_y_list = []
    subpart_y_list.append((range_y[0], midpoint_y))
    subpart_y_list.append((midpoint_y, range_y[1]))

    for subpart_x in subpart_x_list:
        for subpart_y in subpart_y_list:
            size = int((subpart_x[1] - subpart_x[0] + 1)*(subpart_y[1] - subpart_y[0] + 1))
            if size > 0:
                searchUncovered(readingList,subpart_x,subpart_y,size)
            else:
                print(f'Size Error with ranges {range_x} and {range_y} and subparts range_x {subpart_x} and range_y {subpart_y}')
    

##### ROTATE COORDINATES

In [44]:
central_coord = (2_000_000,2_000_000)
range_x = [0, 4_000_000]
range_y = [0, 4_000_000]

#SampleValues
#central_coord = (10,10)
#range_x = [0, 20]
#range_y = [0, 20]


In [45]:
# New plan rotate +45º to have squares instead of losanges
def getRotXRotY(point,central_coord = central_coord):
    d_x = point[0] - central_coord[0]
    d_y = point[1] - central_coord[1]
    rotX = d_x - d_y
    rotY = d_y + d_x
    return rotX, rotY

def getRegXRegY(pointRot, central_coord = central_coord):
    assert (pointRot[0] + pointRot[1]) % 2 == 0
    regX = int((pointRot[0] + pointRot[1])/2 + central_coord[0])
    assert (pointRot[1] - pointRot[0]) % 2 == 0
    regY = int((pointRot[1] - pointRot[0])/2 + central_coord[1])
    return regX, regY

In [46]:
def validateSegment(segment,range_x,range_y,debug = False):
    for pointRot in segment:
        pointReg = getRegXRegY(pointRot, central_coord)
        if debug:
            print(f'    Checking segment point {pointReg}')
        if not (min(range_x) <= pointReg[0] <= max(range_x) and min(range_y) <= pointReg[1] <= max(range_y)):
            if debug:
                print(f'    Segment not valid {segment} because of regPoint {pointReg}')
            return False
    return True

In [47]:
debugInfo = True
for readIndex, reading in enumerate(readingList):
    (rotX, rotY) = getRotXRotY(reading['sensor'], central_coord)
    reading['sensorRot'] = (rotX, rotY)

    # Get the sensor covered boundaries

    distance = reading['distance']

    top_coverage_point = (reading['sensor'][0], reading['sensor'][1] + distance)
    bottom_coverage_point = (reading['sensor'][0], reading['sensor'][1] - distance)
    left_coverage_point = (reading['sensor'][0] - distance, reading['sensor'][1])
    right_coverage_point = (reading['sensor'][0] + distance, reading['sensor'][1])
    
    top_coverage_point_Rot = getRotXRotY(top_coverage_point, central_coord)
    bottom_coverage_point_Rot = getRotXRotY(bottom_coverage_point, central_coord)
    left_coverage_point_Rot = getRotXRotY(left_coverage_point, central_coord)
    right_coverage_point_Rot = getRotXRotY(right_coverage_point, central_coord)

    reading['maxCovRegY'] = top_coverage_point[1]
    reading['minCovRegY'] = bottom_coverage_point[1]
    reading['minCovRegX'] = left_coverage_point[0]
    reading['maxCovRegX'] = right_coverage_point[0]

    reading['maxCovRotY'] = top_coverage_point_Rot[1]
    reading['minCovRotY'] = bottom_coverage_point_Rot[1]
    reading['minCovRotX'] = left_coverage_point_Rot[0]
    reading['maxCovRotX'] = right_coverage_point_Rot[0]

    assert reading['maxCovRotY'] == right_coverage_point_Rot[1]
    assert reading['minCovRotY'] == left_coverage_point_Rot[1]
    assert reading['minCovRotX'] == top_coverage_point_Rot[0]
    assert reading['maxCovRotX'] == bottom_coverage_point_Rot[0]

    # Get the first-non covered row in each direction

    distance = reading['distance'] + 1 # Just ouside the range!

    top_coverage_point = (reading['sensor'][0], reading['sensor'][1] + distance)
    bottom_coverage_point = (reading['sensor'][0], reading['sensor'][1] - distance)
    left_coverage_point = (reading['sensor'][0] - distance, reading['sensor'][1])
    right_coverage_point = (reading['sensor'][0] + distance, reading['sensor'][1])
    
    top_coverage_point_Rot = getRotXRotY(top_coverage_point, central_coord)
    bottom_coverage_point_Rot = getRotXRotY(bottom_coverage_point, central_coord)
    left_coverage_point_Rot = getRotXRotY(left_coverage_point, central_coord)
    right_coverage_point_Rot = getRotXRotY(right_coverage_point, central_coord)

    top_coverage_point_Bound_Fix = 2*max(0, top_coverage_point[1] - range_y[1]) # Limit to boundaries 0 and 4_000_000
    bottom_coverage_point_Bound_Fix = 2*max(0, range_y[0] - bottom_coverage_point[1])
    left_coverage_point_Bound_Fix = 2*max(0, range_x[0] - left_coverage_point[0])
    right_coverage_point_Bound_Fix = 2*max(0,right_coverage_point[0] - range_x[1])


    reading['segmentsRot'] = []
    top_segment = ((top_coverage_point_Rot[0] + top_coverage_point_Bound_Fix, top_coverage_point_Rot[1]),(right_coverage_point_Rot[0] - right_coverage_point_Bound_Fix, right_coverage_point_Rot[1]))
    if debugInfo:
        print(f'Validating Top Segment with top point {top_coverage_point} and fix {top_coverage_point_Bound_Fix}')
        print(f'                      and right point {right_coverage_point} and fix {right_coverage_point_Bound_Fix}')
    if validateSegment(top_segment, range_x, range_y, debugInfo):
        reading['segmentsRot'].append(top_segment)

    right_segment = ((right_coverage_point_Rot[0],right_coverage_point_Rot[1] - right_coverage_point_Bound_Fix),(bottom_coverage_point_Rot[0], bottom_coverage_point_Rot[1] + bottom_coverage_point_Bound_Fix))
    if debugInfo:
        print(f'Validating Right Segment with right point {right_coverage_point} and fix {right_coverage_point_Bound_Fix}')
        print(f'                         and bottom point {bottom_coverage_point} and fix {bottom_coverage_point_Bound_Fix}')
    if validateSegment(right_segment, range_x, range_y, debugInfo):
        reading['segmentsRot'].append(right_segment)

    bottom_segment = ((bottom_coverage_point_Rot[0] - bottom_coverage_point_Bound_Fix,bottom_coverage_point_Rot[1]),(left_coverage_point_Rot[0] + left_coverage_point_Bound_Fix, left_coverage_point_Rot[1]))
    if debugInfo:
        print(f'Validating Bottom Segment with bottom point {bottom_coverage_point} and fix {bottom_coverage_point_Bound_Fix}')
        print(f'                             and left point {left_coverage_point} and fix {left_coverage_point_Bound_Fix}')
    if validateSegment(bottom_segment, range_x, range_y, debugInfo):
        reading['segmentsRot'].append(bottom_segment)

    left_segment = ((left_coverage_point_Rot[0], left_coverage_point_Rot[1] + left_coverage_point_Bound_Fix), (top_coverage_point_Rot[0], top_coverage_point_Rot[1] - top_coverage_point_Bound_Fix))
    if debugInfo:
        print(f'Validating Left Segment with left point {left_coverage_point} and fix {left_coverage_point_Bound_Fix}')
        print(f'                          and top point {top_coverage_point} and fix {top_coverage_point_Bound_Fix}')
    if validateSegment(left_segment, range_x, range_y, debugInfo):
        reading['segmentsRot'].append(left_segment)
    #reading['segmentsRot'] = [top_segment, right_segment, bottom_segment, left_segment]

Validating Top Segment with top point (3988693, 4125554) and fix 251108
                      and right point (4128128, 3986119) and fix 256256
    Checking segment point (4114247, 4000000)
    Segment not valid ((114247, 4114247), (-114247, 4114247)) because of regPoint (4114247, 4000000)
Validating Right Segment with right point (4128128, 3986119) and fix 256256
                         and bottom point (3988693, 3846684) and fix 0
    Checking segment point (4000000, 3857991)
    Checking segment point (3988693, 3846684)
Validating Bottom Segment with bottom point (3988693, 3846684) and fix 0
                             and left point (3849258, 3986119) and fix 0
    Checking segment point (3988693, 3846684)
    Checking segment point (3849258, 3986119)
Validating Left Segment with left point (3849258, 3986119) and fix 0
                          and top point (3988693, 4125554) and fix 251108
    Checking segment point (3849258, 3986119)
    Checking segment point (3863139, 400000

In [48]:
len(readingList), readingList

(24,
 [{'sensor': (3988693, 3986119),
   'beacon': (3979063, 3856315),
   'distance': 139434,
   'sensorRot': (2574, 3974812),
   'maxCovRegY': 4125553,
   'minCovRegY': 3846685,
   'minCovRegX': 3849259,
   'maxCovRegX': 4128127,
   'maxCovRotY': 4114246,
   'minCovRotY': 3835378,
   'minCovRotX': -136860,
   'maxCovRotX': 142008,
   'segmentsRot': [((142009, 3857991), (142009, 3835377)),
    ((142009, 3835377), (-136861, 3835377)),
    ((-136861, 3835377), (-136861, 3863139))]},
  {'sensor': (1129181, 241785),
   'beacon': (1973630, -98830),
   'distance': 1185064,
   'sensorRot': (887396, -2629034),
   'maxCovRegY': 1426849,
   'minCovRegY': -943279,
   'minCovRegX': -55883,
   'maxCovRegX': 2314245,
   'maxCovRotY': -1443970,
   'minCovRotY': -3814098,
   'minCovRotX': -297668,
   'maxCovRotX': 2072460,
   'segmentsRot': [((-297669, -1443969), (2072461, -1443969)),
    ((2072461, -1443969), (2072461, -1927539)),
    ((185901, -3814099), (-185901, -3814099)),
    ((-297669, -3702331

In [49]:
def overlappedRotHorSegment(segment_x ,segment_y,reading):
    assert segment_y[0] == segment_y[1]
    segment_y = segment_y[0]
    segment_x_init, segment_x_end = segment_x

    sensor_x, sensor_y = reading['sensorRot']
    distance = reading['distance']
    sensingRange_x = (reading['minCovRotX'], reading['maxCovRotX'])
    sensingRange_y = (reading['minCovRotY'], reading['maxCovRotY'])

    if not( min(sensingRange_y) <= segment_y <= max(sensingRange_y)):
        #print(f'    Sensor too far {sensor_x, sensor_y}, vertical distance {abs(sensor_y - segment_y)}, maximum distance {distance}')
        return []

    overlap_x_min = max(segment_x_init, min(sensingRange_x))
    overlap_x_max = min(segment_x_end, max(sensingRange_x))
    if overlap_x_min <= overlap_x_max:
        print(f'        Found coverage by sensor {sensor_x,sensor_y} with distance {distance} vertical reach {sensingRange_y} coverage limits {sensingRange_x}')
        return (overlap_x_min, overlap_x_max)
    else:
        return []        

def overlappedRotVertSegment(segment_x ,segment_y, reading):    
    assert segment_x[0] == segment_x[1]    
    segment_x = segment_x[0]
    segment_y_init, segment_y_end = segment_y

    sensor_x, sensor_y = reading['sensorRot']
    distance = reading['distance']
    sensingRange_x = (reading['minCovRotX'], reading['maxCovRotX'])
    sensingRange_y = (reading['minCovRotY'], reading['maxCovRotY'])

    if not( min(sensingRange_x) <= segment_x <= max(sensingRange_x)):
        #print(f'    Sensor too far {sensor_x, sensor_y} from segment horizontal location {segment_x}, horizontal distance {abs(sensor_x - segment_x)}, maximum distance {distance}')
        return []
    
    overlap_y_min = max(segment_y_init,  min(sensingRange_y))
    overlap_y_max = min(segment_y_end,  max(sensingRange_y))
    if overlap_y_min <= overlap_y_max:
        print(f'        Found coverage by sensor {sensor_x,sensor_y} with distance {distance} horizontal reach {sensingRange_x} coverage limits {sensingRange_y}')        
        return (overlap_y_min, overlap_y_max)
    else:
        #print(f'     Out of range bounds {overlap_y_min} and {overlap_y_max}')
        return []

In [50]:
def overlappedRotSegmentList(segment_x, segment_y, readingList):
    hor = False
    vert = False
    overlapList = []
    if segment_x[0] == segment_x[1]:
        print('    Vertical Segment')
        overlapFunc = overlappedRotVertSegment
        vert = True
        print(f'    Segment {segment_y} with horiz {segment_x[0]}')        
    if segment_y[0] == segment_y[1]:
        print('    Horizontal Segment')
        overlapFunc = overlappedRotHorSegment
        hor = True
        print(f'    Segment {segment_x} with heigh {segment_y[0]}')
    if not (hor or vert):
        print('    Unrecognized segment type')
        return [], hor, vert
    for reading in readingList:
        overlap = overlapFunc(segment_x, segment_y, reading)
        if len(overlap) > 0:
            overlapList.append(overlap)

    unifiedOverlapList = unifySegments(overlapList)
    

    return overlapList, hor, vert

In [51]:
def unifySegments(segmentList):
    if not segmentList:
        return []

    pointList = []
    for segment in segmentList:
        pointList.append((min(segment),'begin'))
        pointList.append((max(segment),'end'))
    orderedPoints = sorted(pointList)
    

    coverage_level = 0
    cleanPointList = []
    firstPoint = orderedPoints.pop(0)
    if firstPoint[1] != 'begin':
        print(f'PROBLEM FOUND {segmentList} and {firstPoint}')
    assert firstPoint[1] == 'begin'
    cleanPointList.append(firstPoint)
    coverage_level += 1
    for point in orderedPoints:
        if point[1] == 'begin':
            if coverage_level == 0:
                if len(cleanPointList) == 0:
                    cleanPointList.append(point)
                elif (cleanPointList[-1][1] == 'end') and (cleanPointList[-1][0] == point[0] - 1):
                    print(f'     Unify: removing {cleanPointList[-1]} and not adding {point}')
                    cleanPointList.pop(-1)
                else:
                    cleanPointList.append(point)
            coverage_level += 1
        if point[1] == 'end':
            coverage_level -= 1
            if coverage_level == 0:
                if len(cleanPointList) == 0:
                    cleanPointList.append(point)
                # elif (cleanPointList[-1][1] == 'begin') and (cleanPointList[-1][0] == point[0]):
                #     print(f'     Unify: removing {cleanPointList[-1]} and not adding {point}')
                #     cleanPointList.pop(-1)
                else:
                    cleanPointList.append(point)
    return cleanPointList


In [52]:
for readIndex, reading in enumerate(readingList):

    sensorRotCoord = reading['sensorRot']
    sensingRange_x = (reading['minCovRotX'], reading['maxCovRotX'])
    sensingRange_y = (reading['minCovRotY'], reading['maxCovRotY'])

    print(f'Processing sensor number: {readIndex} with RotCoord {sensorRotCoord} horizontal reach {sensingRange_x} and vertical reach {sensingRange_y}')
    for segmentRot in reading['segmentsRot']:
        #print()
        #print(segmentRot)
        segment_x = (segmentRot[0][0],segmentRot[1][0])
        segment_y = (segmentRot[0][1],segmentRot[1][1])
        segmentOverlapList, segmentHor, segmentVert = overlappedRotSegmentList(sorted(segment_x), sorted(segment_y), readingList)
        print(f'    {segmentOverlapList}')
        unifiedOverlapList = unifySegments(segmentOverlapList)
        print(f'    {unifiedOverlapList}')
        if len(unifiedOverlapList) > 2:
            print('CANDIDATE!!!!')
        if segmentHor:
            if (int(unifiedOverlapList[0][0]) == int(min(segment_x)) and int(unifiedOverlapList[1][0]) == int(max(segment_x))):
                print('    Fully Covered')
            else:
                print(f'FOUND HOR!!!!!!!!')
        if segmentVert:
            if (int(unifiedOverlapList[0][0]) == int(min(segment_y)) and int(unifiedOverlapList[1][0]) == int(max(segment_y))):
                print('    Fully Covered')
            else:
                print('FOUND VERT!!!!!!!!')
        
        

Processing sensor number: 0 with RotCoord (2574, 3974812) horizontal reach (-136860, 142008) and vertical reach (3835378, 4114246)
    Vertical Segment
    Segment [3835377, 3857991] with horiz 142009
        Found coverage by sensor (-192701, 3129455) with distance 705923 horizontal reach (-898624, 513222) coverage limits (2423532, 3835378)
        Found coverage by sensor (579708, 3408598) with distance 456960 horizontal reach (122748, 1036668) coverage limits (2951638, 3865558)
    [(3835377, 3835378), (3835377, 3857991)]
    [(3835377, 'begin'), (3857991, 'end')]
    Fully Covered
    Horizontal Segment
    Segment [-136861, 142009] with heigh 3835377
        Found coverage by sensor (-164090, 3835400) with distance 286838 vertical reach (3548562, 4122238) coverage limits (-450928, 122748)
        Found coverage by sensor (-192701, 3129455) with distance 705923 vertical reach (2423532, 3835378) coverage limits (-898624, 513222)
        Found coverage by sensor (579708, 3408598) wit

In [53]:
foundSolutionRot = (-14731,2423531)
foundSolution =getRegXRegY(foundSolutionRot)
foundSolution

(3204400, 3219131)

In [54]:
range_x[1]*foundSolution[0] + foundSolution[1]

12817603219131